# Customizing your workflow
In the previous tutorial we saw how to load the data you create in a simulation. Here we'll go into some more detail about how to customize your simulations. 

Let's start with imports:

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os

## Overview

The file `examples/run.sh` calls `barriers/utils/workflow.py`, which runs a set of calculations in series (relaxed scan, conformer generation, etc.). The script loads simulation details from the file `job_info.json`. Let's take a look at that file:

In [3]:
path = "../examples/job_info.json"
with open(path, 'r') as f:
    info = json.load(f)
    
display(info)

{'weightpath': '../models/singlet/0',
 'triplet_weightpath': '../models/triplet/0',
 'smiles_list': ['CCN(CC)c1ccc(/N=N\\c2ccc(NC(=O)C[N+](CC)(CC)CC)cc2)cc1',
  'CCN(c1ccccc1)c1cc(C(=O)NOC)ccc1/N=N/c1ccc(C(=O)NOC)cc1N(CC)c1ccccc1'],
 'device': 2,
 'rdkit_confgen': {},
 'relaxed_scan': {},
 'confgen': {'num_parallel': 10, 'num_in_chunk': 100},
 'evf': {'num_parallel': 1},
 'hessian': {'num_parallel': 2},
 'irc': {'num_parallel': 1},
 'triplet_crossing': {'num_parallel': 1}}

Here's what each key means:

- `weightpath`: The directory in which the singlet model is saved. The model should be saved with the name `best_model` in the directory `weightpath`
- `triplet_weightpath`: Same as `weightpath`, but for the triplet model
- `smiles_list`: List of SMILES strings for the molecules you want to analyze
- `device`: Where to run calculations. Either an integer for the GPU you want to use or `cpu` if you don't have a GPU.
- `rdkit_confgen`: Any custom details for the RDKit conformer generation stage
- `relaxed_scan`: Any custom details for the relaxed scan generation stage
- `confgen`: Any custom details for the conformer generation stage
- `evf`: Any custom details for the eigenvector following generation stage
- `hessian`: Any custom details for the Hesssian calculation on the cis and strans geometries
- `irc`: Any custom details for the intrinsic reaction coordinate stage


### Parallelization

In this example, we've set `num_parallel` for many of the stages. This tells you how many jobs you want to run in parallel at a time (e.g. for 2 molecules and 4 mechanisms, you'll have 8 TSs and 4 endpoints, for a total of 12 conformer generation jobs, so you may want to run all of those in parallel). 

#### Details of parallelization
The simplest way to parallelize jobs is to run $N$ separate job scripts, one for each job, and have them all use the same GPU. However, we found that the more jobs you run this way, the slower each job becomes. In fact, past $N\approx 3$, the ratio $(\text{number of jobs}) \ / \ (\text{total time})$ stops increasing.

This can be fixed by running one script only, then putting the geometries from all the calculations into one batch, and evaluating the model on that batch. We implemented this batching approach for conformer generation since it was by far the slowest stage in our workflow. For that reason, you can get very good speedups up until `num_parallel` $\approx 50$ in conformer generation. Since the other methods use separate job scripts for parallelization, we recommend setting `num_parallel`$=4$ for those jobs.

Lastly, note that conformer generation has a second parallelization key, called `num_in_chunk`. Batched conformer generation runs metadynamics and optimizations for each species until its lowest-energy conformer stops decreasing in energy. It then replaces this species with another species that's waiting in the queue. Once all species are done, it performs a final tight optimization on each. `num_in_chunk` is the number of species in the queue. For example, if you had 300 species and `num_in_chunk`$=100$, you'd completely finish the first 100 species before moving starting the next 200.

## Specific job details

The script we're running calls sub-scripts, one for each stage of the workflow, which are located in `scripts`. We can see that here:


In [4]:
display(os.listdir('../scripts'))
display(os.listdir('../scripts/relaxed_scan'))

['irc',
 'relaxed_scan',
 'confgen',
 'hessian',
 'triplet_crossing',
 'full_workflow',
 'rdkit_confgen',
 'evf']

['batch.sh', 'job.sh', 'test', 'default_details.json']

We can see that `scripts` has a sub-folder for each component of the workflow. Each sub-folder has job scripts (`job.sh` for a single job and `batch.sh` for a batched job), a `test` folder for testing the code, and `default_details.json`. This last file has the default details for the job. By looking at those details, we can see what parameters can be customized. You can customize any parameter by setting its value in `job_info.json` as described above.

Now we'll look at those details for each stage of the workflow.

### `rdkit_confgen`

This stage generates initial conformer guesses using a [script in the Neural Force Field repository](https://github.com/learningmatter-mit/NeuralForceField/blob/master/nff/utils/confgen.py) that calls [RDKit](https://www.rdkit.org/docs/GettingStartedInPython.html). Let's look at the default details:

In [5]:
path_template = '../scripts/{name}/default_details.json'
path = path_template.format(name='rdkit_confgen')

with open(path, 'r') as f:
    details = json.load(f)
    
display(details)

{'csv_data_path': 'smiles.csv',
 'max_confs': 200,
 'forcefield': 'mmff',
 'nconf_gen': 2000,
 'e_window': 5.0,
 'rms_tol': 0.1,
 'prun_tol': 0.01,
 'job_dir': 'confs',
 'log_file': 'confgen.log',
 'rep_e_window': 5.0,
 'fallback_to_align': False,
 'temp': 298.15,
 'pickle_save_dir': '.',
 'summary_save_dir': '.',
 'clean_up': True}

Here's what each key means:

- `csv_data_path`: If you're running this script by itself, it'll look for a CSV path with a list of SMILES strings. Our workflow script generates this file automatically.
- `max_confs`: Maximum number of conformers to keep at the very end
- `forcefield`: Classical force field to use for optimization
- `nconf_gen`: How many initial conformers to generate (duplicates and high-energy conformers will get discarded)
- `e_window`: Only keep conformers with relative energy below `e_window`, in kcal/mol
- `rms_tol`: Conformers with RMSD below `rms_tol` Angstroms are considered duplicates and discarded after optimization.
- `prun_tol`: Same idea as `rms_tol`, but for the initial conformer generation by RDKit.
- `job_dir`: Where to save the `xyz` files and scratch work from conformer generation
- `log_file`: Where to log notes and errors
- `rep_e_window`: Only log information about conformers with energies under `rep_e_window` kcal/mol
- `fallback_to_align`: Use `obabel --align` if `obfit` fails
- `temp`: Temperature in Kelvin for computing statistical weights of each conformer
- `pickle_save_dir`: Directory in which to save the pickle file with information about each conformer
- `summary_save_dir`: Directory in which to save the json file with a summary of the results, without the actual conformers themselves
- `clean_up`: Remove scratch information once the job is done

### `relaxed_scan`
This stage performs a relaxed scan from an initial conformer guess to a TS guess. Let's look at the default details:

In [6]:
with open(path_template.format(name='relaxed_scan'), 'r') as f:
    details = json.load(f)
    
display(details)

{'nxyz': None,
 'smiles': None,
 'weightpath': None,
 'num_parallel': 4,
 'nnid': '',
 'en_key': 'energy_0',
 'model_kwargs': None,
 'device': 0,
 'cutoff': 5.0,
 'cutoff_skin': 2.0,
 'nbr_list_update_freq': 10,
 'directed': True,
 'requires_large_offsets': False,
 'opt_config': 'neuraloptimizer',
 'opt_max_step': 500,
 'fmax': 0.05,
 'fmax_tight': 0.05,
 'num_steps': 20,
 'opt_type': 'BFGS',
 'use_rdkit': False,
 'end_constraints': {'hookean': {'atoms': {'idx': None,
    'template_smiles': 'c1ccc(/N=N/c2ccccc2)cc1',
    'targets': None,
    'force_consts': 2242.34},
   'bonds': {'idx': None,
    'template_smiles': 'c1ccc(/N=N/c2ccccc2)cc1',
    'targets': None,
    'force_consts': 2242.34},
   'angles': {'idx': None,
    'template_smiles': 'c1ccc(/N=N/c2ccccc2)cc1',
    'targets': None,
    'force_consts': 627.5},
   'dihedrals': {'idx': None,
    'template_smiles': 'c1ccc(/N=N/c2ccccc2)cc1',
    'targets': None,
    'force_consts': 627.5}}}}

Here's what each key means:

- `nxyz`: Coordinates of the starting geometry. Provided automatically in the workflow script from the previous RDKit stage
- `smiles`: SMILES string of the species. Provided automatically in the workflow script
- `weightpath`: Directory to model
- `num_parallel`: Number of jobs to run in parallel
- `nnid`: Can join the strings `weightpath` and `nnid` to give the model directory if you want
- `en_key`: Name of the energy key outputted by the model
- `model_kwargs`: Any keyword arguments you want to provide to the model when calling it
- `device`: Device to run the calculation on
- `cutoff`: Neighbor list cutoff for the model
- `cutoff_skin`: Use `cutoff` $+$ `cutoff_skin` as the cutoff when generating neighbors, and then only take neighbors within `cutoff` of each other. This is done because we only recompute the neighbors every few steps, so we don't want to miss any if an atom comes into another atoms' neighborhood between updates.
- `nbr_list_update_freq`: Number of steps between neighbor list updates in the optimization
- `directed`: Use a directed neighbor list (must be the case for PaiNN)
- `requires_large_offsets`: Use large offsets for periodic structures (not relevant for us)
- `opt_config`: Name of the script to use for the optimization at each stage of the scan
- `opt_max_step`: Maximum number of optimization steps
- `fmax`: Maximum force threshold for convergence optimization ($\mathrm{eV} \ / \ A$).
- `fmax_tight`: Same as `fmax`, but for a more tight optimization on the last geometry after the scan is complete. Make sure to use the same value as for `fmax_tight` in conformer generation (see below).
- `num_steps`: Number of steps to use in the relaxed scan
- `opt_type`: Name of the ASE optimization engine to use
- `use_rdkit`: Use RDKit instead of ASE to adjust the atom positions to their target values at each step
- `end_constraints`: Constraints to apply to the geometry at the last stage of the scan. The values at intermediate steps are interpolated between the starting values and the final values.
    - The constraint dictionary has keys for each type of constraint (see [here](https://wiki.fysik.dtu.dk/ase/ase/constraints.html) for a list of built-in ASE constraints). We use `hookean`, which provides a spring force to keep things at their target values.
    - Within `hookean`, we can specify constraints to keep atoms at certain positions (`atoms`), to keep bonds at certain lengths (`bonds`) or to keep angles/dihedral angles at certain values (`angles`/`dihedrals`)
    - Whichever coordinate you constrain, you have to supply `idx`, a list of indices for the coordinates of interest. For example:
         - `"atoms": [0, 6, 9]`; `"bonds": [[3, 4], [5, 6], [7, 8]]`; `"angles": [[3, 4, 9], [6, 7, 9]]`; `"dihedrals": [[3, 4, 9, 10], [0, 3, 4, 5], [1, 2, 3, 4]]`
    - You can also ask to use the indices from a reference molecule, given by `template_smiles`. The program then does a substructure search to figure out what those indices are in the current molecule. 
        - This is what we do in the workflow: we set `template_smiles` to `'c1ccc(/N=N/c2ccccc2)cc1'` (azobenzene), and then supply the indices as `[3, 4, 5, 6]` for the dihedrals (these are the CNNC atoms in azobenzene), and/or  `[3, 4, 5]` and `[4, 5, 6]` for the angles
    - Lastly, you must supply force constants in units of kcal or kcal/A.

    


### `confgen`
This stage performs conformer generation on the TS guesses from the relaxed scans, and also on the RDKit guesses for the *cis* and *trans* geometries. Let's look at the default details:

In [7]:
with open(path_template.format(name='confgen'), 'r') as f:
    details = json.load(f)
    
display(details)

{'weightpath': None,
 'smiles': None,
 'nxyz': None,
 'num_parallel': 10,
 'num_in_chunk': 100,
 'nnid': '',
 'en_key': 'energy_0',
 'model_kwargs': None,
 'device': 0,
 'cutoff': 5.0,
 'cutoff_skin': 2.0,
 'nbr_list_update_freq': 10,
 'directed': True,
 'requires_large_offsets': False,
 'md_type': 'NoseHooverMetaDynamics',
 'geom_add_time': 1000,
 'mtd_time': None,
 'infer_time_from_flex': True,
 'time_step': 2.0,
 'temperature': 298.15,
 'ttime': 50,
 'maxwell_temp': None,
 'loginterval': 10,
 'fixed_atoms': {'idx': [3, 4, 5, 6],
  'template_smiles': 'c1ccc(/N=N/c2ccccc2)cc1'},
 'constraints': {'hookean': {'bonds': {'idx': None,
    'template_smiles': None,
    'targets': None,
    'force_consts': 2242.34},
   'angles': {'idx': None,
    'template_smiles': None,
    'targets': None,
    'force_consts': 627.5},
   'dihedrals': {'idx': None,
    'template_smiles': None,
    'targets': None,
    'force_consts': 627.5}}},
 'opt_constraints': None,
 'enhanced_sampling': {'method': 'NoseHo

Here's what each key means (other than the ones we already defined above):
- `md_type`: What kind of molecular dynamics (MD) to run to sample geometries
- `geom_add_time`: How many fs between adding new geometries to the biasing potential
- `mtd_time`: Total metadynamics time, in ps. Given as `None` because we use `infer_from_flex`
- `infer_time_from_flex`: Use a molecule flexibility measure to determine the metadynamics time
- `time_step`: Time step, in fs
- `temperature`: Temperature in Kelvin
- `ttime`: $\tau= \mathrm{ttime} \cdot \mathrm{time\_step}$ is the Nose-Hoover relaxation time
- `maxwell_temp`: Temperature with which to generate initial velocities from Maxwell-Boltzmann distribution. Defaults to $2\cdot$`temperature` if not specified
- `loginterval`: Log MD statistics every `loginterval` steps
- `fixed_atoms`: Dictionary telling you which atoms to **fix** (not constrain) during dynamics and optimization. For TS conformer searches, we constrain the CNNC atoms, by setting `idx` to `[3, 4, 5, 6]` and using azobenzene as a template. For cis/trans searches we don't fix these atoms, but we do exclude them from the metadynamics bias (see below)

- `constraints`: Any constraints you want to use. Same format as in `relaxed_scan` 
- `opt_constraints`: Any constraints you want to apply during the optimization portion but not during dynamics
- `enhanced_sampling`: Dictionary with information about the enhanced sampling method
    - `method`: The method name
    - `params`: Any parameters that need to be given for the enhanced sampling method. For metadynamics we specify the number of turn on steps $\kappa$, the pushing strength $k_i' = k_i / N$, where $N$ is the number of atoms and $k_i'$ is in mHa, the Gaussian width $\alpha_i$ (in units of $\mathrm{Bohr}^{-2}$), the bias type (RMSD by default), and the maximum number of reference structures to use in the pushing potential
    - `shake`: Whether to use SHAKE to constrain bond lengths

- `exclude_from_rmsd`: Any atoms to exclude from the RMSD computation for the pushing potential in metadynamics. We set these to `[3, 4, 5, 6]` with azobenzene as the template SMILES. For TS searches this is redundant, since fixing `[3, 4, 5, 6]` will automatically exclude them from the RMSD. But it is necessary for *cis* and *trans* optimizations, so that we don't accidentally turn *cis* into *trans* and vice-versa.

- `sample_rate_fs`: Sample geometries every `sample_rate_fs` fs from dynamics to optimize as conformers
- `lower_e_tol`: Tolerance for energy changes, in kcal/mol. If the energy drops by less than `lower_e_tol` after a round of metadynamics + optimization, then no more rounds are performed for that species

- `max_md_runs`: maximum number of metadynamics runs
- `min_md_runs`: minimum number of metadynamics runs

- `fmax_coarse`: Force tolerance during coarse optimization, in $\mathrm{eV} \ / \ A$
- `fmax_tight`: Force tolerance during tight optimization, in $\mathrm{eV} \ / \ A$
- `fmax_vtight`: Force tolerance during very tight optimization, in $\mathrm{eV} \ / A$
    - In the paper we used 0.01 for TSs and 0.005 for *cis*/*trans*, and then used 0.005 for EVF. Here for simplicity we've used 0.005 for all, so that we can use the same parameters for all conformer jobs.
- `window_coarse`: Only keep conformers with energy below this value during coarse optimization. Given in $\mathrm{kcal} \ /  \ \mathrm{mol}$
- `window_tight`: Only keep conformers with energy below this value during tight optimization. Given in $\mathrm{kcal} \ /  \ \mathrm{mol}$
- `window_vtight`: Only keep conformers with energy below this value during very tight optimization. Given in $\mathrm{kcal} \ /  \ \mathrm{mol}$


- `opt_type`: Name of the optimization engine. We implemented our own custom batched optimizers for BFGS and LBFGS. LBFGS is significantly faster because it does not diagonalize Hessian-related matrices. BFGS becomes very slow when diagonalizing such matrices for all geometries in the batch.

- `opt_max_step`: Maximum number of steps in the optimization
- `check_hess`: Check Hessian for imaginary frequencies to see if an optimized structure is really a minimum
- `max_restart_opt`: Maximum number of optimization restarts if the initial optimization didn't give a true minimum 
- `crest_dedupe`: Dictionary with information about removing duplicate conformers using `cregen` in the CREST program
    - `on`: Whether or not to use this functionality
    - `params`: Any parameters for `cregen`. See the [cregen documentation](https://xtb-docs.readthedocs.io/en/latest/crestcmd.html#ensemble-sorting) for more details on these parameters


### `evf`
This stage performs eigenvector following on the TS conformers from `confgen`. Let's look at the default details:

In [8]:
with open(path_template.format(name='evf'), 'r') as f:
    details = json.load(f)
    
display(details)

{'weightpath': None,
 'nxyz': None,
 'num_parallel': 4,
 'nnid': '',
 'device': 0,
 'ev_kwargs': {'maxstepsize': 0.15,
  'maxstep': 1000,
  'convergence': 0.005,
  'method': 'Powell'},
 'atoms_kwargs': {'cutoff': 5.0,
  'cutoff_skin': 2.0,
  'nbr_update_period': 1,
  'directed': True,
  'requires_large_offsets': False},
 'calc_kwargs': {'en_key': 'energy_0',
  'properties': ['energy', 'forces'],
  'model_kwargs': None},
 'thermo_kwargs': {'flip_all_but_ts': True, 'imag_cutoff': -10},
 'cutoff_to_be_ts': -200,
 'confs_per_ts': 5}

Here's what each key means (other than the ones we already defined above):
- `ev_kwargs`: A dictionary with parameters for the eigenvector following routine:
    - `maxstepsize`: Maximum step size, in Angstroms
    - `convergence`: Same as `fmax`, in $\mathrm{eV} \ / A $
    - `method`: Method for updating the Hessian. The default is the Powell method 
- `atoms_kwargs`: A dictionary with parameters for the `AtomsBatch` objects (basically our Neural Force Field version of ASE atoms objects). All keys in this dictionary were defined above; in other methods they existed inside the main dictionary.
- `calc_kwargs`: A dictionary with parameters for the ASE calculator using the model. Most keys in this dictionary were defined above; in other methods they existed inside the main dictionary. The only new key is `properties`, which tells you what you want to calculate (energies, forces, or both)
- `thermo_kwargs`: Information about quantities in thermodynamic calculations:
    - `flip_all_but_ts`: Turn all but the lowest imaginary frequency into a positive frequency
    - `imag_cutoff`: Any imaginary frequency with magnitude under `imag_cutoff` $\mathrm{cm}^{-1}$ can be flipped and considered real
- `cutoff_to_be_ts`: If the lowset imaginary frequency has magnitude under `cutoff_to_be_ts` $\mathrm{cm}^{-1}$, then it is not considered to be a true TS
- `confs_per_ts`: How many TS conformers to optimize with eigenvector following

### `hessian`
This stage performs Hessian calculations on the confgen-optimized *cis* and *trans* conformers. Let's look at the default details:

In [9]:
with open(path_template.format(name='hessian'), 'r') as f:
    details = json.load(f)
    
display(details)

{'weightpath': None,
 'nxyz': None,
 'num_parallel': 4,
 'nnid': '',
 'atoms_kwargs': {'cutoff': 5.0,
  'cutoff_skin': 2.0,
  'nbr_update_period': 1,
  'directed': True,
  'requires_large_offsets': False},
 'calc_kwargs': {'en_key': 'energy_0',
  'properties': ['energy', 'forces'],
  'model_kwargs': None},
 'device': 0,
 'analytical_hessian': False,
 'temperature': 298.15,
 'pressure': 101325,
 'imag_cutoff': -10,
 'rotor_cutoff': 50,
 'confs_per_endpoint': 1}

Here's what each key means (other than the ones we already defined above):

- `analytical_hessian`: Calculate the Hessian analytically. This is turned off by default because it takes up a lot of memory for bigger molecules. To avoid unexpected out-of-memory errors, we compute the Hessian numerically with ASE
- `pressure`: Pressure for the thermo calculation, in Pa
- `rotor_cutoff`: Cutoff in $\mathrm{cm}^{-1}$ used in the interpolation between harmonic oscillator and rigid rotor treatments of low-frequency modes
- `confs_per_endpoint`: How many conformers to perform Hessian calculations on

### `irc`
This stage performs intrinsic reaction coordinate (IRC) calculations on optimized TSs. Let's look at the default details:

In [10]:
with open(path_template.format(name='irc'), 'r') as f:
    details = json.load(f)
    
display(details)

{'nxyz': None,
 'weightpath': None,
 'num_parallel': 4,
 'nnid': '',
 'device': 0,
 'en_key': 'energy_0',
 'model_kwargs': None,
 'init_displ_de': 0.000398,
 'max_iter': 5000,
 'tol_max_g': 0.002,
 'tol_rms_g': 0.0005,
 'scale_displ_sd_corr': 0.3333,
 'sd_corr_parabolic_fit': True,
 'do_sd_corr': True,
 'interpolate_only': True,
 'sd_parabolic_fit': True,
 'adapt_scale_disp': True,
 'scale_displ_sd': 0.15,
 'cutoff': 5.0,
 'cutoff_skin': 2.0,
 'nbr_update_period': 1,
 'directed': True,
 'requires_large_offsets': False,
 'confs_per_ts': 1}

Here's what each key means (other than the ones we already defined above):

- `init_displ_de`: Target energy change after the first step in the direction of the imaginary mode. Given in Ha
- `max_iter`: Maximum number of steps to generate the path
- `tol_max_g`: Maximum gradient tolerance for convergence (same as `fmax`), given in $\mathrm{Ha} \ / \ \mathrm{Bohr}$
- `rms_max_g`: RMS gradient tolerance for convergence (same as `fmax`), given in $\mathrm{Ha} \ / \ \mathrm{Bohr}$
- `scale_displ_sd_corr`: Factor for scaling the correction to the steepest descent (SD) step
- `sd_corr_parabolic_fit`: Do a parabolic fit for finding the optimal SD step length 
- `do_sd_corr`: Apply a correction to the first SD step
- `interpolate_only`: Only allow interpolation for parabolic fit, not extrapolation
- `sd_parabolic_fit`: Do a parabolic fit for finding the optimal SD step length
- `adapt_scale_disp`: Modify `scale_displ_sd` when the step size becomes smaller or larger
- `scale_displ_sd`: Factor for scaling the first SD step
- `confs_per_ts`: How many EVF-optimized TS conformers to perform IRC on


### `triplet_crossing`
This stage performs a search for the lowest energy singlet-triplet crossing on either side of the rotational TS. The initial guesses are generated by generating a forward- and backward-IRC path, and looking for crossings along each. Let's look at the default details:

In [11]:
with open(path_template.format(name='triplet_crossing'), 'r') as f:
    details = json.load(f)
    
display(details)

{'nxyz': None,
 'nnid': '',
 'weightpath': None,
 'num_parallel': 4,
 'triplet_params': {'nnid': '',
  'weightpath': None,
  'en_key': 'energy_0',
  'model_kwargs': None,
  'cutoff': 5.0},
 'triplet_nnid': '',
 'init_displ_de': 0.000398,
 'max_iter': 500,
 'tol_max_g': 0.0001,
 'tol_rms_g': 1e-05,
 'scale_displ_sd_corr': 0.3333,
 'sd_corr_parabolic_fit': True,
 'do_sd_corr': True,
 'interpolate_only': True,
 'sd_parabolic_fit': True,
 'adapt_scale_disp': True,
 'scale_displ_sd': 0.01,
 'alpha_kcal': 12.55,
 'opt_type': 'BFGS',
 'fmax': 0.005,
 'opt_max_step': 1500,
 'nbr_list_update_freq': 10,
 'en_key': 'energy_0',
 'model_kwargs': None,
 'cutoff': 5.0,
 'device': 0,
 'cutoff_skin': 2.0,
 'directed': True,
 'requires_large_offsets': False,
 'compute_hessian': False,
 'temperature': 298.15,
 'pressure': 101325,
 'imag_cutoff': -10,
 'rotor_cutoff': 50,
 'k_isc_params': {'h_so_inv_cm': 20, 'method': 'wkb_analytical'}}

Here's what each key means (other than the ones we already defined above):

- `triplet_params`: Parameters related to calculations of the triplet enegries and forces. By default no `weightpath` is given, but if `triplet_weightpath` is specified in the workflow JSON, then it'll automatically get put into `triplet_params-
- `triplet_nnid`: `nnid` for the triplet model
- `alpha_kcal`: $\alpha$ parameter for the singlet-triplet optimization (see SI of the paper). Given in kcal/mol
- `compute_hessian`: Compute singlet and triplet Hessians at the crossing points, and get the associated thermostatistical quantities. By default we don't do this and just use the quantities from the TS that generated the crossing points
- `k_isc_params`: Parameters for computing the intersystem crossing rate:
    - `h_so_inv_cm`: The spin-orbit coupling strength $H_{\mathrm{SO}}$, given in $\mathrm{cm^{-1}}$. By default we take this to be $20 \ \mathrm{cm^{-1}}$ for all derivatives
    - `method`: The method used to calculate the rate. By default we use `wkb_analytical`, which computes the rate using the analytical method based on the WKB approximation, from [this paper](https://pubs.acs.org/doi/full/10.1021/jp503794j?casa_token=E7gsTA6AG9sAAAAA%3AcYFKUI4PZn1E1rC7f5ULN4p1hMXFPhj_TUb8iYm3XNp8Ut9w-Tf0wP_Uv4dxe9Qz4XCi2eHkll8P_Nj1)